In [6]:
!pip install pyspark streamlit matplotlib seaborn plotly pyngrok dask

In [7]:
! pip install openai langchain langchain-community chromadb tiktoken transformers sentence-transformers torch

In [8]:
# Main Streamlit App
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

# === UI Setup ===
page_bg_color = """
<style>
[data-testid="stAppViewContainer"] {
    background-color: azure;
}
</style>
"""
st.markdown(page_bg_color, unsafe_allow_html=True)

# Initialize Spark
spark = SparkSession.builder.appName("FoodTradeAnalysis").getOrCreate()

# Load data
df = spark.read.csv("food_trade_data.csv", header=True, inferSchema=True)
pdf = df.toPandas()

# Load population data
population_df_usa = pd.read_csv("population_usa.csv")
population_df_global = pd.read_csv("population.csv")

# Sidebar Navigation
st.sidebar.title("Go-to")
page = st.sidebar.radio("Go to", ["Home", "Country Insights", "Chat Bot"])

# ===== HOME PAGE =====
if page == "Home":
    st.title("🌐 Global Food Supply Chain")
    st.markdown("#### Explore worldwide import/export patterns in the food sector using visual analytics.")

    st.subheader("📈 Trade Trends Over Time")
    line_data = pdf.groupby(["Year", "Country"])[["Import_Value_USD", "Export_Value_USD"]].sum().reset_index()
    line_data = line_data.melt(id_vars=["Year", "Country"], var_name="Trade_Type", value_name="Value_USD")
    trend_fig = px.line(line_data, x="Year", y="Value_USD", color="Country", line_dash="Trade_Type", title="Global Import/Export Trends Over Time")
    st.plotly_chart(trend_fig)

    st.subheader("🏷️ Country Comparison by Food Category")
    bar_data = pdf.groupby(["Country", "Category"])[["Import_Value_USD", "Export_Value_USD"]].sum().reset_index()
    bar_data["Total_Value"] = bar_data["Import_Value_USD"] + bar_data["Export_Value_USD"]
    bar_fig = px.bar(bar_data, x="Country", y="Total_Value", color="Category", barmode="group", title="Total Food Trade by Country and Category")
    st.plotly_chart(bar_fig)

    st.subheader("📊 Total Imports by Country")
    imports = pdf.groupby("Country")["Import_Value_USD"].sum().reset_index().sort_values(by="Import_Value_USD", ascending=False)
    import_fig = px.bar(imports, x="Country", y="Import_Value_USD", color="Import_Value_USD", color_continuous_scale="Reds", title="Total Import Values by Country")
    st.plotly_chart(import_fig)

    st.subheader("📊 Total Exports by Country")
    exports = pdf.groupby("Country")["Export_Value_USD"].sum().reset_index().sort_values(by="Export_Value_USD", ascending=False)
    export_fig = px.bar(exports, x="Country", y="Export_Value_USD", color="Export_Value_USD", color_continuous_scale="Greens", title="Total Export Values by Country")
    st.plotly_chart(export_fig)

    st.subheader("🕸️ Radar Plot: Category-wise Trade Comparison (Top 5 Countries)")
    top_countries = pdf.groupby("Country")[["Import_Value_USD", "Export_Value_USD"]].sum()
    top_countries["Total_Trade"] = top_countries["Import_Value_USD"] + top_countries["Export_Value_USD"]
    top5 = top_countries.sort_values("Total_Trade", ascending=False).head(5).index.tolist()

    radar_data = pdf[pdf["Country"].isin(top5)].copy()
    radar_data["Total_Trade"] = radar_data["Import_Value_USD"] + radar_data["Export_Value_USD"]
    radar_pivot = radar_data.groupby(["Country", "Category"])["Total_Trade"].sum().reset_index()
    radar_df = radar_pivot.pivot(index="Country", columns="Category", values="Total_Trade").fillna(0)
    radar_df_norm = radar_df.div(radar_df.max(axis=1), axis=0)

    import plotly.graph_objects as go
    radar_fig = go.Figure()
    for country in radar_df_norm.index:
        radar_fig.add_trace(go.Scatterpolar(
            r=radar_df_norm.loc[country].values,
            theta=radar_df_norm.columns,
            fill='toself',
            name=country
        ))
    radar_fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), showlegend=True, title="Top 5 Countries: Normalized Category-wise Trade Radar")
    st.plotly_chart(radar_fig)

    st.subheader("🗺️ Global Food Trade Map by Year")
    for year in [2020, 2021, 2022, 2023, 2024]:
        map_data = pdf[pdf["Year"] == year].copy()
        map_data["Total_Trade"] = map_data["Import_Value_USD"] + map_data["Export_Value_USD"]
        st.markdown(f"**🌍 {year} Trade Map**")
        map_fig = px.choropleth(map_data, locations="Country", locationmode="country names", color="Total_Trade", color_continuous_scale="plasma", title=f"Total Food Trade Values by Country ({year})")
        st.plotly_chart(map_fig)

# ===== COUNTRY INSIGHTS =====
elif page == "Country Insights":
    st.title("📊 Country-wise Food Trade Dashboard")
    countries = pdf["Country"].unique()
    selected_country = st.sidebar.selectbox("Select a Country", countries)

    filtered_data = pdf[pdf["Country"] == selected_country]

    st.subheader(f"📦 {selected_country}: Import and Export by Food Category")
    category_data = filtered_data.groupby("Category")[["Import_Value_USD", "Export_Value_USD"]].sum().reset_index()
    category_data = category_data.melt(id_vars=["Category"], var_name="Trade Type", value_name="Value")
    category_fig = px.bar(category_data, x="Category", y="Value", color="Trade Type", barmode="group", text_auto=True,
                          color_discrete_map={"Import_Value_USD": "#1f77b4", "Export_Value_USD": "#17becf"},
                          title=f"{selected_country}: Import vs Export by Food Category")
    category_fig.update_layout(xaxis_tickangle=-45, title_x=0.2)
    st.plotly_chart(category_fig)

    st.subheader(f"📈 {selected_country}: Per Capita Import & Export Value Over Years")
    if selected_country == "United States":
        pop_df = population_df_usa
    else:
        pop_df = population_df_global[population_df_global["Country"] == selected_country]

    per_capita_df = filtered_data.merge(pop_df, on=["Country", "Year"], how="left")
    per_capita_df["Per_Capita_Import"] = per_capita_df["Import_Value_USD"] / per_capita_df["Population"]
    per_capita_df["Per_Capita_Export"] = per_capita_df["Export_Value_USD"] / per_capita_df["Population"]
    melt_df = per_capita_df[["Year", "Per_Capita_Import", "Per_Capita_Export"]].melt(id_vars="Year", var_name="Trade Type", value_name="Per Capita Value (USD)")
    melt_df["Trade Type"] = melt_df["Trade Type"].replace({
        "Per_Capita_Import": "Per Capita Import",
        "Per_Capita_Export": "Per Capita Export"
    })
    capita_fig = px.line(melt_df, x="Year", y="Per Capita Value (USD)", color="Trade Type", markers=True, title=f"{selected_country}: Per Capita Import & Export Trend")
    capita_fig.update_layout(title_x=0.3, plot_bgcolor="rgba(0,0,0,0)")
    st.plotly_chart(capita_fig)

    st.markdown("### Visualizing trends across food categories and years")
    chart_type = st.radio("Select chart type:", ["Trends", "Bar", "Per Capita"], horizontal=True)

    if chart_type == "Trends":
        fig = px.line(filtered_data, x="Year", y=["Import_Value_USD", "Export_Value_USD"],
                      color_discrete_map={"Import_Value_USD": "red", "Export_Value_USD": "green"},
                      title=f"{selected_country} - Total Import vs Export Trends", markers=True)
    elif chart_type == "Bar":
        fig = px.bar(filtered_data, x="Year", y=["Import_Value_USD", "Export_Value_USD"],
                     barmode="group", title=f"{selected_country} - Total Import vs Export Bar View")
    else:
        if selected_country == "United States":
            pop_df = population_df_usa
        else:
            pop_df = population_df_global[population_df_global["Country"] == selected_country]
        per_capita_df = filtered_data.merge(pop_df, on=["Country", "Year"], how="left")
        per_capita_df["Per_Capita_Import"] = per_capita_df["Import_Value_USD"] / per_capita_df["Population"]
        per_capita_df["Per_Capita_Export"] = per_capita_df["Export_Value_USD"] / per_capita_df["Population"]
        melt_df = per_capita_df[["Year", "Per_Capita_Import", "Per_Capita_Export"]].melt(
            id_vars="Year", var_name="Trade Type", value_name="Per Capita Value (USD)")
        melt_df["Trade Type"] = melt_df["Trade Type"].replace({
            "Per_Capita_Import": "Per Capita Import",
            "Per_Capita_Export": "Per Capita Export"
        })
        fig = px.bar(melt_df, x="Year", y="Per Capita Value (USD)", color="Trade Type", barmode="group",
                     title=f"{selected_country} - Per Capita Import & Export Over Years")

    fig.update_layout(title_x=0.3, plot_bgcolor="rgba(0,0,0,0)")
    st.plotly_chart(fig)

    st.subheader("📦 Category-wise Import vs Export")

    for cat in filtered_data["Category"].unique():
        temp = filtered_data[filtered_data["Category"] == cat]

        # Merge population based on selected country
        if selected_country == "United States":
            pop_df = population_df_usa
        else:
            pop_df = population_df_global[population_df_global["Country"] == selected_country]

        temp = temp.merge(pop_df, on=["Country", "Year"], how="left")
        temp["Per_Capita_Import"] = temp["Import_Value_USD"] / temp["Population"]
        temp["Per_Capita_Export"] = temp["Export_Value_USD"] / temp["Population"]

        fig_type = st.radio(
            f"{cat} View Type:",
            ["Trends", "Bar", "Per Capita"],
            key=f"{cat}_view"
        )

        fig, ax = plt.subplots()
        if fig_type == "Trends":
            ax.plot(temp["Year"], temp["Import_Value_USD"], label="Import", color="red")
            ax.plot(temp["Year"], temp["Export_Value_USD"], label="Export", color="green")
            ax.set_ylabel("USD Value")
        elif fig_type == "Bar":
            ax.bar(temp["Year"] - 0.2, temp["Import_Value_USD"], width=0.4, label="Import", color="red")
            ax.bar(temp["Year"] + 0.2, temp["Export_Value_USD"], width=0.4, label="Export", color="green")
            ax.set_ylabel("USD Value")
        else:  # Per Capita View
            ax.bar(temp["Year"] - 0.2, temp["Per_Capita_Import"], width=0.4, label="Per Capita Import", color="orange")
            ax.bar(temp["Year"] + 0.2, temp["Per_Capita_Export"], width=0.4, label="Per Capita Export", color="blue")
            ax.set_ylabel("Per Capita USD Value")

        ax.set_title(f"{cat} - {fig_type} View")
        ax.set_xlabel("Year")
        ax.legend()
        st.pyplot(fig)


    st.subheader("📊 Heatmap: Category vs Year")
    pivot_import = filtered_data.pivot_table(index="Category", columns="Year", values="Import_Value_USD", aggfunc='sum')
    pivot_export = filtered_data.pivot_table(index="Category", columns="Year", values="Export_Value_USD", aggfunc='sum')

    st.markdown("#### Import Heatmap")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(pivot_import, cmap="Reds", ax=ax)
    st.pyplot(fig)

    st.markdown("#### Export Heatmap")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(pivot_export, cmap="Greens", ax=ax)
    st.pyplot(fig)

# ===== CHAT BOT =====
elif page == "Chat Bot":
    st.title("🤖 Chat Bot for Food Trade Insights")

    from langchain_community.vectorstores import Chroma
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.llms import HuggingFacePipeline
    from langchain.document_loaders import DataFrameLoader
    from langchain.text_splitter import CharacterTextSplitter
    from langchain.chains import RetrievalQA
    from transformers import pipeline

    data = pdf.fillna("N/A")
    loader = DataFrameLoader(data, page_content_column="Country")
    docs = loader.load()
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(split_docs, embedding=embeddings)

    flan_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256, do_sample=True, top_k=30, top_p=0.95)
    llm = HuggingFacePipeline(pipeline=flan_pipeline)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

    static_context = (
        "Context:\n"
        "- Overall, Processed food is the highest imported/exported food.\n"
        "- China is the highest importing country.\n"
        "- Brazil is the highest exporting country.\n"
        "- Australia is the least importing country.\n"
        "- Japan is the least exporting country.\n\n"
    )

    if "history" not in st.session_state:
        st.session_state.history = []

    user_question = st.text_input("You:", key="input")

    if user_question:
        combined_prompt = static_context + "Question: " + user_question
        response = qa_chain.run(combined_prompt)
        st.session_state.history.append({"user": user_question, "bot": response})

    for chat in reversed(st.session_state.history):
        st.markdown(f"**You:** {chat['user']}")
        st.markdown(f"**Bot:** {chat['bot']}")


Overwriting app.py


In [9]:
!curl ipv4.icanhazip.com


34.87.57.149


In [10]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼your url is: https://eager-socks-vanish.loca.lt
^C
